# Test Generation on Google Colab
Generate tests for bugs using GPU acceleration

In [ ]:
# Mount Google Drive for persistence
from google.colab import drive
drive.mount('/content/drive')

# Set working directory
import os
os.chdir('/content/drive/MyDrive/libro_replication')

In [ ]:
# Install dependencies
!pip install transformers torch accelerate huggingface_hub sentencepiece -q
!pip install tqdm pyyaml -q

In [ ]:
# Check GPU
import torch
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'None'}")

In [ ]:
# Clone/pull repository code
# (Assumes you've uploaded your code to Drive)
import sys
sys.path.append('/content/drive/MyDrive/libro_replication')

In [ ]:
# Login to HuggingFace
from huggingface_hub import notebook_login
notebook_login()

## Run Generation

In [ ]:
import json
import logging
from src.model_manager import ModelManager
from src.core.prompt_builder import PromptBuilder
from src.core.test_generator import TestGenerator
from src.core.batch_processor import BatchProcessor

logging.basicConfig(level=logging.INFO)

In [ ]:
# Load bug reports
with open('data/bugs/bug_reports.json') as f:
    bug_reports = json.load(f)

print(f"Loaded {len(bug_reports)} bug reports")

In [ ]:
# Initialize model (use StarCoder2-15B on A100)
model_manager = ModelManager(
    model_key="starcoder2-15b",
    cache_dir="models/starcoder2-15b"
)
model_manager.load()

In [ ]:
# Initialize components
prompt_builder = PromptBuilder(
    num_examples=2,
    examples_file="data/examples/manual_examples.json"
)

test_generator = TestGenerator(
    model_manager=model_manager,
    cache_dir="cache/generations"
)

batch_processor = BatchProcessor(
    prompt_builder=prompt_builder,
    test_generator=test_generator,
    output_dir="results/batch_30"
)

In [ ]:
# Process bugs (30 bugs, 10 samples each)
results = batch_processor.process_bugs(
    bug_reports=bug_reports[:30],
    n_samples=10
)

In [ ]:
# View results summary
print(f"\nProcessed {len(results)} bugs")
print(f"Total tests generated: {sum(len(tests) for tests in results.values())}")

# Show sample
for bug_id, tests in list(results.items())[:3]:
    print(f"\n{bug_id}: {len(tests)} tests")
    if tests:
        print(tests[0]['test_code'][:150])